In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os


In [3]:
load_dotenv(dotenv_path = "password.env") 

# MONGO_URI example: MONGO_URI=mongodb://localhost:27017/

mongo_uri = os.getenv("mongo_uri")
mongo_database = os.getenv("mongo_database")

client = MongoClient(mongo_uri)

db = client[mongo_database]

In [60]:
# Loading in csv data
patrons_data = pd.read_csv('patrons.csv')
inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
inventory['isbn'] = inventory['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')


# Converting to dictionaries
patrons_dict = patrons_data.to_dict('records')
loans_dict = loans.to_dict('records')
books_dict = books.to_dict('records')
inventory_dict = inventory.to_dict('records')
authors_dict = authors.to_dict('records')

# Adding authors as a subset of books
for item in books_dict:
    item['author'] = {}
for item in authors_dict:
    author_id = item['author_id']
    for book in books_dict:
        if book['author_id'] == author_id:
            book['author'] = item

# Merging books' meta data with inventoried books
for item in books_dict:
    isbn = item['isbn']
    for book in inventory_dict:
        if book['isbn'] == isbn:
            book['title'] = item['title']
            book['pub_year'] = item['pub_year']
            book['genre'] = item['genre']
            book['overdue_fee_rate'] = item['overdue_fee_rate']
            book['author'] = item['author']

# Adding inventories books as a subset of loans
for item in loans_dict:
    item['book'] = {}
for item in inventory_dict:
    book_id = item['book_id']
    for loan in loans_dict:
        if loan['book_ID'] == book_id:
            loan['book'] = item
    item.pop('book_id')

# Adding loans as a subset of patrons
for item in patrons_dict:
    item['loan_history'] = []
for item in loans_dict:
    patron_id = item['patron_ID']
    for patron in patrons_dict:
        if patron['patron_id'] == patron_id:
            patron['loan_history'].append(item)
            item.pop("patron_ID")
            item.pop("loan_ID")
            item.pop("book_ID")
            break

# Getting rid of `patron_id` variable in patrons
for item in patrons_dict:
    item.pop('patron_id', None)


In [ ]:
# PATRONS COLLECTION

# Adding patron objects to MongoDB
patrons = db.patrons
patrons.insert_many(patrons_dict)

# Delete current objects to repopulate if necessary
# patrons.delete_many({})

[{'first_name': 'Arjuna',
  'last_name': 'Bazaz',
  'address': '111 Main Street',
  'phone_number': '123-456-7891',
  'age': 21,
  'loan_history': [{'loan_date': '2025-01-09',
    'due_date': '2025-02-08',
    'return_date': '2025-01-25',
    'days_overdue': 0.0,
    'book': {'isbn': '9780812981602',
     'condition': 'Fair',
     'title': 'The Vegetarian',
     'pub_year': 2007,
     'genre': 'Literary Fiction',
     'overdue_fee_rate': 0.5,
     'author': {'author_id': 29,
      'first_name': 'Han',
      'last_name': 'Kang',
      'nationality': 'South Korean',
      'language': 'Korean',
      'birth_year': '1970',
      'biography': "Han Kang wrote 'The Vegetarian,' which won the Man Booker International Prize."}}}],
  '_id': ObjectId('683727a160f44cae1186aae6')},
 {'first_name': 'JT',
  'last_name': 'Torres',
  'address': '112 Main Street',
  'phone_number': '198-765-4321',
  'age': 20,
  'loan_history': [{'loan_date': '2024-08-16',
    'due_date': '2024-09-15',
    'return_date'

In [ ]:
# BOOKS COLLECTION

# Adding book objects to MongoDB
books = db.books
books.insert_many(inventory_dict)

# Delete current objects to repopulate if necessary
# books.delete_many({})

[{'first_name': 'Arjuna',
  'last_name': 'Bazaz',
  'address': '111 Main Street',
  'phone_number': '123-456-7891',
  'age': 21,
  'loan_history': [{'loan_date': '2025-01-09',
    'due_date': '2025-02-08',
    'return_date': '2025-01-25',
    'days_overdue': 0.0,
    'book': {'isbn': '9780812981602',
     'condition': 'Fair',
     'title': 'The Vegetarian',
     'pub_year': 2007,
     'genre': 'Literary Fiction',
     'overdue_fee_rate': 0.5,
     'author': {'author_id': 29,
      'first_name': 'Han',
      'last_name': 'Kang',
      'nationality': 'South Korean',
      'language': 'Korean',
      'birth_year': '1970',
      'biography': "Han Kang wrote 'The Vegetarian,' which won the Man Booker International Prize."},
     '_id': ObjectId('68372b3960f44cae1186abb1')}}]},
 {'first_name': 'JT',
  'last_name': 'Torres',
  'address': '112 Main Street',
  'phone_number': '198-765-4321',
  'age': 20,
  'loan_history': [{'loan_date': '2024-08-16',
    'due_date': '2024-09-15',
    'return_da